<a href="https://colab.research.google.com/github/LalchandShivraj/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

In [10]:
# create df by filtering the dataset where the total_votes >= 20 pick reviews.
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vote_count_df = df.filter("total_votes >= 20")


In [11]:
# create a new df by filtering the preceding df where helpful_votes/total_votes is >= 50%
votes_df = vote_count_df.filter(vote_count_df["helpful_votes"]/vote_count_df["total_votes"]>=0.5)
votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42943632|R2VIQ3UX794Q0O|B00YMNSCYM|      74738477|LE LED Under Cabi...|Home Improvement|          4|           21|         21|   N|                Y|Nice Kit, I showe...|Nice Kit, I showe...| 2015-08-31|
|         US|   24649241|R1OA24IIHWF54G|B00YI5TNXI|     813068315|Hyperikon BR20-Se...|Home Improvement|          5|    

In [12]:
# create a new df by filtering where a review was written as part of the Vine program(paid), vine =='Y'
vine_program_df = votes_df.filter(votes_df["vine"] == "Y")
vine_program_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   53093730|R29V4UDSS053D8|B00P6ORMR4|     406381545|Aquasana AQ-RO-3....|Home Improvement|          3|           33|         38|   Y|                N|Not for average h...|This filter is ov...| 2015-08-30|
|         US|   25984637|R1I2D068WC37PA|B00ZWBURWI|     302288140|Myfox Security Ca...|Home Improvement|          3|    

In [13]:
# filter the votes_df for rows not in the Vine Program.
not_vine_program_df = votes_df.filter(votes_df["vine"] == "N")
not_vine_program_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42943632|R2VIQ3UX794Q0O|B00YMNSCYM|      74738477|LE LED Under Cabi...|Home Improvement|          4|           21|         21|   N|                Y|Nice Kit, I showe...|Nice Kit, I showe...| 2015-08-31|
|         US|   24649241|R1OA24IIHWF54G|B00YI5TNXI|     813068315|Hyperikon BR20-Se...|Home Improvement|          5|    

In [14]:
# Determine the total number of reviews, the number of 5-star reviews, 
# and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

# number of reviews:
votes_df.count()

39095

In [15]:
# df with r star reviews

# number of 5-star reviews:
df_five_star = votes_df.filter(votes_df["star_rating"]== 5)
df_five_star.count()

18371

In [18]:
# percentage of 5-star reviews for PAID review type.
paid_votes_df = votes_df.filter(votes_df["verified_purchase"]== "Y")
(df_five_star.filter(df_five_star["verified_purchase"] == "Y").count()/paid_votes_df.count()) * 100

49.67071839837429

In [19]:
# percentage of 5-star reviews for UNPAID review type.
unpaid_votes_df = votes_df.filter(votes_df["verified_purchase"]== "N")
(df_five_star.filter(df_five_star["verified_purchase"] == "N").count()/unpaid_votes_df.count()) * 100

41.30330618112122